<a href="https://colab.research.google.com/github/Hideki-Iwaki-TUS/Seminer2025/blob/Mei/ch07_StatisticalArbitrage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# パッケージのインポート

In [ ]:
#numpy:数値計算 pandas:データ操作　matplotlib.pyplot:グラフ描画　をインポート。
#時系列分析のJohansen共和分検定とベクトル誤差修正モデル（VECM）のためにstatsmodelsライブラリの特定のモジュールをインポート
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.vector_ar.vecm import coint_johansen
from statsmodels.tsa.api import VECM

これらは、後続のコードで時系列データの分析とモデル構築を行うために必要なツールです。

**import pandas as pd**

このコード は、Pythonでデータ操作や分析によく使われる pandas ライブラリをインポートしています。pd という短いエイリアスを付けることで、コード内で pandas の機能を使う際に pd.function_name のように簡潔に記述できるようになります。

**import matplotlib.pyplot as plt**

このコードは、Pythonでグラフを作成したりデータを可視化するためによく使われる matplotlib.pyplot モジュールをインポートしています。plt という短いエイリアスを付けることで、コード内で matplotlib.pyplot の機能を使う際に plt.function_name のように簡潔に記述できるようになります。


**from statsmodels.tsa.vector_ar.vecm import coint_johansen**


これは、共和分過程かどうかを統計的に検定する「Johansen共和分検定」を実行するための coint_johansen 関数をインポートしている。

**from statsmodels.tsa.api import VECM**

VECMは、パラメーターを推定する。


# パラメータの設定

In [ ]:
Sigma = np.array([[1,0,0],[0,1,0],[0,0,1]])

a_1 = np.array([0.1,0.2,0.3]) # alpha_1
a_2 = np.array([0.5,-0.3,-0.3])
b_1 = np.array([1,-0.5,-0.5]) # beta_1
b_2 = np.array([-0.2,0.5,-0.2])

A = np.stack([a_1,a_2],axis = 1) # alpha_2
B = np.stack([b_1,b_2]) # beta_2

Pi_1 = np.dot(a_1.reshape(3,1),b_1.reshape(1,3))
Pi_2 = np.dot(A,B)


はい、このコードは時系列モデル、特にVECMのパラメータを定義しています。

np.array は NumPy ライブラリの関数で、NumPy配列（ndarray）を作成するために使用されます。NumPy配列は、数値計算を効率的に行うための多次元配列オブジェクトです。


Sigma: 誤差項の共分散行列を表す3x3の単位行列です。

a_1, a_2: 「アルファ」ベクトルを表します。

b_1, b_2: 「ベータ」ベクトルを表します。

A, B: a_1とa_2をスタックして行列A（アルファ行列）を作成し、b_1とb_2をスタックして行列B（ベータ行列）を作成しています。

axis=0（デフォルト）: 新しい軸を先頭（0番目）に追加して結合します。例えば、2つの1次元配列を axis=0 で結合すると、それぞれの1次元配列が行となり、2次元配列が作成されます。
axis=1: 新しい軸を1番目に追加して結合します。例えば、2つの1次元配列を axis=1 で結合すると、それぞれの1次元配列が列となり、2次元配列が作成されます。

Pi_1: a_1とb_1の外積として計算されます。これは、特定の共和分関係を持つ長期的な影響行列を示します。

Pi_2: AとBの行列積として計算されます。これは、より高次の共和分ランクを持つ可能性のある別の共和分行列を表しています。
これらのパラメータは、後続のシミュレーションやVECMの推定に使用されます。

a_1 は np.array([0.1,0.2,0.3]) という1次元配列（形状は (3,)）です。要素が３つある１次元配列

a_1.reshape(3,1) はこの1次元配列を 3行1列の2次元配列（列ベクトル） に変換します。つまり、[[0.1], [0.2], [0.3]] のようになります。


同様に、b_1 は np.array([1,-0.5,-0.5]) という1次元配列（形状は (3,)）です。


b_1.reshape(1,3) はこの1次元配列を 1行3列の2次元配列（行ベクトル） に変換します。つまり、[[1, -0.5, -0.5]] のようになります。

reshape:配列の形状（次元）を変更するため

In [ ]:
#乱数を発生させる
np.random.seed(seed = 1)
rand_nums = np.random.randn(20000,3)

乱数を生成するための準備を行っている。


**np.random.seed(seed = 1)**

これはNumPyの乱数生成器のシードを設定しています。シード（初期値）を設定することで、コードを何回実行しても常に同じ乱数列が生成されるようになります。これにより、結果の再現性が保証されます。


**rand_nums = np.random.randn(20000,3)**

np.random.randn() は、標準正規分布（平均0、標準偏差1）に従う乱数を生成する関数です。
(20000, 3) は、生成される配列の形状を指定しています。これは、20,000行、3列の2次元配列を意味します。つまり、3つの変数に対する正規分布に従うランダムなノイズや誤差を表すデータセットが作成されます。



In [ ]:
#S_??の初期化
init = np.array([1,1,1])
S_rw = init #ランダムウォーク
S_ci1 = init　#ランク１の共和分過程
S_ci2 = init　#ランク２の共和分過程

変数の初期値設定。

**init = np.array([1,1,1])**

これは init という名前の変数に、要素が [1, 1, 1] のNumPy配列を代入しています。この配列は、シミュレーションを開始する際の基準となる初期状態を表します。

**S_rw = init**

**S_ci1 = init**

**S_ci2 = init**

これらの行では、S_rw、S_ci1、S_ci2 という3つの変数を、先に定義した init 配列で初期化しています。これは、異なる種類の時系列モデル（例えば、ランダムウォークや共和分モデル）のシミュレーションを開始する際の共通の初期条件として、すべて [1, 1, 1] からスタートすることを示しています。


In [ ]:
#時系列の生成
for rand_num in rand_nums:
    S_rw = np.vstack((S_rw,S_rw[len(S_rw)-1] + np.dot(rand_num,Sigma)))
    S_ci1 = np.vstack((S_ci1,np.dot(Pi_1 + np.eye(3),S_ci1[len(S_ci1)-1])
                       + np.dot(rand_num,Sigma)))
    S_ci2 = np.vstack((S_ci2,np.dot(Pi_2 + np.eye(3),S_ci2[len(S_ci2)-1])
                       + np.dot(rand_num,Sigma)))



事前に設定されたパラメータと乱数を使用して、3種類の時系列データをシミュレーションしています。

**for rand_num in rand_nums**

これは、乱数配列 rand_nums の各行（つまり各時点のランダムショック＝予測できない突然の変動や外乱を表す要素。ホワイトノイズ？）を順に処理するためのループです。

各時系列 S_rw, S_ci1, S_ci2 の更新は、以下の論理に基づいて行われます。

**S_rw = np.vstack((S_rw,S_rw[len(S_rw)-1] + np.dot(rand_num,Sigma)))**

これはランダムウォークをシミュレーションしています。次の時点の S_rw の値は、現在の S_rw の値に、乱数 rand_num と共分散行列 Sigma の積（つまりランダムなショック）を加えたものとして計算されます。
np.vstack は、新しい行（計算された次の時点の値）を既存の S_rw 配列の下に垂直方向に結合して追加しています。

S_rw[len(S_rw)-1]

これは S_rw 配列の現在の長さを取得し、そこから1を引くことで、最後の要素のインデックスを指しています。つまり、現在の時系列の最後の時点の値を参照するために len が使用されています。


**S_ci1 = np.vstack((S_ci1,np.dot(Pi_1 + np.eye(3),S_ci1[len(S_ci1)-1]) + np.dot(rand_num,Sigma)))**

これは共和分ランク1の時系列をシミュレーションしています。np.eye(3)（3x3の単位行列）が加えられています。この部分は、前の状態と長期的な関係に基づいて決定される変動を表します。


これに rand_num と Sigma から計算されるランダムなショックが加算され、S_ci1 が更新されます。


**S_ci2 = np.vstack((S_ci2,np.dot(Pi_2 + np.eye(3),S_ci2[len(S_ci2)-1]) + np.dot(rand_num,Sigma)))**

これは共和分ランク2の時系列をシミュレーションしています。基本的な構造は S_ci1 と同じですが、ここでは Pi_2 行列が使用されています。これにより、異なる共和分関係がモデル化されます。


まとめると、このループはそれぞれの初期値から開始し、20,000ステップにわたってランダムウォークおよび共和分関係を持つ時系列データを生成しています。これにより、後のJohansen検定やVECMの推定で利用されるデータが作成されます。

# Johansen検定

## $\mathbf{S}_{\textbf{RW}}$をテスト(共和分検定を)する

In [ ]:
#ランダムウォークに対して共和分過程
JohansenTestResult_rw = coint_johansen(S_rw,k_ar_diff=0,det_order=-1)

**coint_johansen** は、複数の非定常な時系列変数間に長期的な安定した関係（共和分関係）が存在するかどうかを統計的に検定するための関数です。

S_rw は、理論的には共和分関係がないと予想されます。


**k_ar_diff=0**

これは、階数の個数が０と指定。0は、VECMモデルにラグがないことを意味します。

**det_order=-1**

これは、共和分検定における決定論的トレンド成分（切片やトレンド項）の扱いを指定。

-1 は、定数項もトレンド項もモデルに含まないことを意味します。定数項ｃをゼロに制約、つまり、モデルはデータが原点から始まることを仮定します。

トレンド項（trend term）とは、時系列データに長期的に見られる持続的な上昇または下降の動き、つまり「トレンド」を表現するための数学的な要素のことです。時系列分析において、データが時間とともに一方向に変化する傾向を捉えるためにモデルに組み込まれます。


この検定の結果は JohansenTestResult_rw に格納される

出力


In [ ]:
print(JohansenTestResult_rw.lr1) # Trace statistic

[22.58036266  8.78820583  2.43645402]


lr1 は、ヨハンセン共和分検定における「トレース統計量（Trace statistic）」を指します。トレース統計量は、共和分関係の数（共和分ランク）を判断するために使われる統計量の一つです。

具体的には、共和分ランクが0から最大数までであるという帰無仮説を順次検定するために使用されます。この値と、同時に出力される臨界値（critical values）を比較することで、共和分関係が存在するかどうか、そしてその数がいくつであるかを統計的に判断します。

In [ ]:
print(JohansenTestResult_rw.lr2) # Maximum eigenvalue statistic

[13.79215683  6.35175181  2.43645402]


lr2 は、ヨハンセン共和分検定における「最大固有値統計量（Maximum eigenvalue statistic）」を指します。最大固有値統計量も、トレース統計量と同様に共和分関係の数（共和分ランク）を判断するために使われる統計量の一つです。

この統計量は、共和分ランクが r であるという帰無仮説を、共和分ランクが r+1 であるという対立仮説に対して検定するために使用されます。この値と、同時に出力される臨界値（critical values）を比較することで、共和分関係が存在するかどうか、そしてその数がいくつであるかを統計的に判断します。



In [ ]:
print(JohansenTestResult_rw.cvt) # Critical values (90%,95%,99%) of trace statistic

[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]


In [ ]:
print(JohansenTestResult_rw.lr2) # Maximum eigenvalue statistic

[13.79215683  6.35175181  2.43645402]


In [ ]:
print(JohansenTestResult_rw.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


## $\mathbf{S}_{\textbf{CI},1}$をテストする

In [ ]:
JohansenTestResult_s1 = coint_johansen(S_ci1,k_ar_diff=0,det_order=-1)

In [ ]:
print(JohansenTestResult_s1.lr1) # Trace statistic
print(JohansenTestResult_s1.cvt) # Critical values (90%,95%,99%) of trace statistic
print(JohansenTestResult_s1.lr2) # Maximum eigenvalue statistic
print(JohansenTestResult_s1.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[1.15097289e+04 1.42336157e+01 1.08408520e+00]
[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]
[1.14954952e+04 1.31495305e+01 1.08408520e+00]
[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


## $\mathbf{S}_{\textbf{CI,2}}$をテストする

In [ ]:
JohansenTestResult_s2 = coint_johansen(S_ci2,k_ar_diff=0,det_order=-1)

In [ ]:
print(JohansenTestResult_s2.lr1) # Trace statistic
print(JohansenTestResult_s2.cvt) # Critical values (90%,95%,99%) of trace statistic
print(JohansenTestResult_s2.lr2) # Maximum eigenvalue statistic
print(JohansenTestResult_s2.cvm) # Critical values (90%,95%,99%) of maximum eigenvalue statistic

[3.24178966e+04 6.11019131e+03 2.57568342e+00]
[[21.7781 24.2761 29.5147]
 [10.4741 12.3212 16.364 ]
 [ 2.9762  4.1296  6.9406]]
[2.63077053e+04 6.10761563e+03 2.57568342e+00]
[[15.7175 17.7961 22.2519]
 [ 9.4748 11.2246 15.0923]
 [ 2.9762  4.1296  6.9406]]


# モデル・パラメータの推定

## $\mathbf{S}_{\textbf{CI},1}$のパラメータ推定

In [ ]:
model_s1 = VECM(S_ci1,k_ar_diff=0,coint_rank = 1, deterministic='na')
res_s1 = model_s1.fit()


In [ ]:
print(res_s1.summary())

                 Loading coefficients (alpha) for equation y1                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.0998      0.003     32.514      0.000       0.094       0.106
                 Loading coefficients (alpha) for equation y2                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.2077      0.003     68.038      0.000       0.202       0.214
                 Loading coefficients (alpha) for equation y3                 
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ec1            0.3019      0.003     98.418      0.000       0.296       0.308
          Cointegration relations for loading-coeffi

In [ ]:
print(res_s1.alpha)
print(res_s1.beta)

[[0.09983864]
 [0.20767309]
 [0.30188794]]
[[ 1.        ]
 [-0.50000816]
 [-0.49972569]]


## $\mathbf{S}_{\textbf{CI},2}$のパラメータ


*   リスト項目
*   リスト項目



In [ ]:
model_s2 = VECM(S_ci2,k_ar_diff=0,coint_rank = 2, deterministic='na')
res_s2 = model_s2.fit()

print(res_s2.alpha)
print(res_s2.beta)

[[ 0.00295261  0.19832263]
 [ 0.26430882 -0.25658122]
 [ 0.36024384 -0.29875471]]
[[ 1.00000000e+00 -1.10297307e-16]
 [ 4.27951128e-17  1.00000000e+00]
 [-8.74973589e-01 -7.49993120e-01]]


In [ ]:
print(np.dot(res_s2.alpha,res_s2.beta.transpose()))
print(Pi_2)


[[ 0.00295261  0.19832263 -0.15132406]
 [ 0.26430882 -0.25658122 -0.03882909]
 [ 0.36024384 -0.29875471 -0.09113987]]
[[ 0.    0.2  -0.15]
 [ 0.26 -0.25 -0.04]
 [ 0.36 -0.3  -0.09]]
